## Accelerate Inference: Neural Network Pruning

In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.layers import *

print(tf.version.VERSION)


2.15.0


In [2]:
# untar
# !tar -xvzf dataset.tar.gz
# load train
train_images = pickle.load(open('train_images.pkl', 'rb'))
train_labels = pickle.load(open('train_labels.pkl', 'rb'))
# load val
val_images = pickle.load(open('val_images.pkl', 'rb'))
val_labels = pickle.load(open('val_labels.pkl', 'rb'))

In [3]:
# Define the neural network architecture (don't change this)

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

In [4]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 25, 32)        896       
                                                                 
 activation (Activation)     (None, 25, 25, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 23, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 23, 23, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 11, 11, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 11, 11, 32)        0         
                                                        

In [5]:
# you can use the default hyper-parameters for training,
# val accuracy ~72% after 50 epochs

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, weight_decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

# history = model.fit(train_images, train_labels, batch_size=32, epochs=50,
#                     validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

In [6]:
results = model.evaluate(val_images, val_labels, batch_size=128)
# model.save('local_model.keras')



20/20 [==============================] - 1s 14ms/step - loss: 1.6110 - accuracy: 0.2075


In [ ]:
import copy
def GetSparsity(model):
  temp_weights = copy.deepcopy(model.get_weights())
  num_weights = 0
  num_nonzeros = 0
  for t in temp_weights:
    num_weights += np.prod(t.shape)
    num_nonzeros += np.count_nonzero(t)


  sparse = 1 - num_nonzeros/num_weights
  return sparse

In [81]:
def magnitude_prune(train_images, train_labels, val_images, val_labels,model):
  batch_size = 32
  num_epoch = 0
  num_increments = 10

  A = copy.deepcopy(model.weights)
  sparsity = []
  accuracy = []
  rand_idxs = np.arange(train_images.shape[0])
  num_batches = int(np.ceil(train_images.shape[0]/batch_size))

  for i in range(num_increments):
    all_masks = {}
    for j in [8, 10]:
      max_mag = np.max(np.abs(A[j]))
      thresh = i/num_increments*max_mag
      mask = np.array(np.abs(A[j]) > thresh).astype(int)
      all_masks[j] = mask

    np.random.shuffle(rand_idxs)
    model.set_weights(A)
    B = copy.deepcopy(model.weights)
    for key in all_masks:
      B[key] = B[key]*all_masks[key]
    model.set_weights(B)

    for j in range(num_epoch):
      print(i, 'epoch', j)
      # for k in range(num_batches):
      #   if k == num_batches-1:
      #     batch_x = train_images[rand_idxs[k*batch_size:]]
      #     batch_y = train_labels[rand_idxs[k*batch_size:]]
      #   else:
      #     batch_x = train_images[rand_idxs[k*batch_size: (k+1)*batch_size]]
      #     batch_y = train_labels[rand_idxs[k*batch_size: (k+1)*batch_size]]
      model.fit(train_images, train_labels, batch_size = 32, verbose = 0)

      #   model.fit(batch_x, batch_y, verbose = 0)
      B = copy.deepcopy(model.weights)
      for key in all_masks:
        B[key] = B[key]*all_masks[key]
      model.set_weights(B)
      #   del B, batch_x, batch_y
      # model.evaluate(val_images, val_labels)
    B = copy.deepcopy(model.weights)
    for key in all_masks:
      B[key] = B[key]*all_masks[key]
    model.set_weights(B)
    del B
    _, acc = model.evaluate(val_images, val_labels)
    sparsity.append(GetSparsity(model))
    accuracy.append(acc)
  return (sparsity, accuracy)


In [59]:
import copy
def progressive_magnitude(train_images, train_labels, val_images, val_labels,model):
  batch_size = 32
  num_iterations = 7
  step_size = 0.1

  A = copy.deepcopy(model.weights)
  sparsity = []
  accuracy = []
  rand_idxs = np.arange(train_images.shape[0])
  num_batches = int(np.ceil(train_images.shape[0]/batch_size))

  original_loss = model.evaluate(train_images, train_labels)
  original_loss = original_loss[0]
  print('Original Loss', original_loss)

  for i in range(num_iterations):
    all_masks = {}
    for j in [8, 10]:
      max_mag = np.max(np.abs(model.weights[j]))
      thresh = i*step_size*max_mag
      mask = np.array(np.abs(model.weights[j]) > thresh).astype(int)
      all_masks[j] = mask

    np.random.shuffle(rand_idxs)
    B = copy.deepcopy(model.weights)
    for key in all_masks:
      B[key] = B[key]*all_masks[key]
    model.set_weights(B)
    del B
    new_loss = model.evaluate(train_images, train_labels, verbose = 0)
    new_loss = new_loss[0]
    print('Init Loss after pruning: ', new_loss)
    print('Sparsity: ', GetSparsity(model))
    j = 0
    while new_loss > (1.1 + i*0.1)*original_loss:
      print(i, 'epoch', j)
      for k in range(num_batches):
        if k == num_batches-1:
          batch_x = train_images[rand_idxs[k*batch_size:]]
          batch_y = train_labels[rand_idxs[k*batch_size:]]
        else:
          batch_x = train_images[rand_idxs[k*batch_size: (k+1)*batch_size]]
          batch_y = train_labels[rand_idxs[k*batch_size: (k+1)*batch_size]]
          
        model.fit(batch_x, batch_y, verbose = 0)
        B = copy.deepcopy(model.weights)
        for key in all_masks:
          B[key] = B[key]*all_masks[key]
        model.set_weights(B)
        del B, batch_x, batch_y
      # model.fit(train_images, train_labels, epochs=1, batch_size = 32)
      # B = copy.deepcopy(model.weights)
      # for key in all_masks:
      #   B[key] = B[key]*all_masks[key]
      # model.set_weights(B)
      # del B
      new_loss = model.evaluate(train_images, train_labels)
      new_loss = new_loss[0]
      j += 1
      model.save_weights(f"my_model_weights_{i}.h5") 

    # B = copy.deepcopy(model.weights)
    # for key in all_masks:
    #   B[key] = B[key]*all_masks[key]
    # model.set_weights(B)
    # del B
    # _, acc = model.evaluate(val_images, val_labels)
    # sparsity.append(GetSparsity(model))
    # accuracy.append(acc)
  # return (sparsity, accuracy)

In [82]:
# perform pruning here
model = keras.models.load_model('./local_model.keras')
# model.evaluate(val_images, val_labels)
# sparsity, acc = magnitude_prune(train_images, train_labels, val_images, val_labels, model)
# np.savetxt('record_noTraining.csv',np.array([sparsity, acc]).T, delimiter=',')

progressive_magnitude(train_images, train_labels, val_images, val_labels, model)

79/79 [==============================] - 0s 4ms/step - loss: 1.6132 - accuracy: 0.1925


In [67]:
print(GetSparsity(model))
print([np.count_nonzero(x) for x in model.weights])

0.8852922674231322
[864, 32, 9216, 32, 18432, 64, 36864, 64, 1624, 512, 305, 5]


In [57]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 25, 32)        896       
                                                                 
 activation (Activation)     (None, 25, 25, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 23, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 23, 23, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 11, 11, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 11, 11, 32)        0         
                                                        

In [62]:
# evaluate again to see how the accuracy changes
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 10ms/step - loss: 0.8400 - accuracy: 0.6851


In [ ]:
np.savetxt('record2.txt',np.array([sparsity, acc]).T)

In [26]:
# you need to save the model's weights, naming it 'my_model_weights.h5'
model.save_weights("my_model_weights.h5")

# running this cell will immediately download a file called 'my_model_weights.h5'
# from google.colab import files
# files.download("my_model_weights.h5")

ModuleNotFoundError: No module named 'google.colab'